# Match markers in two VCF files

Applications such as haplotyping and imputation need to match the markers in a target VCF file to those in a reference panel in a VCF file. Matching can be by marker ID, if the two data sets are on different builds, or by chromosome position, if the two data sets are on the same build.

## Example files

We need example VCF files for demonstation. We can manually download the target VCF file from the [link](http://faculty.washington.edu/browning/beagle/test.08Jun17.d8b.vcf.gz) (877KB) and put it in current working directory. Or, within Julia,

In [7]:
isfile("test.08Jun17.d8b.vcf.gz") || download("http://faculty.washington.edu/browning/beagle/test.08Jun17.d8b.vcf.gz", 
    joinpath(pwd(), "test.08Jun17.d8b.vcf.gz"));
stat("test.08Jun17.d8b.vcf.gz")

StatStruct(mode=0o100644, size=876514)

We can manually download the reference panel for chromosome 22 form the [link](http://bochet.gcc.biostat.washington.edu/beagle/1000_Genomes_phase1_vcf/chr22.1kg.ref.phase1_release_v3.20101123.vcf.gz) (51.5MB) and put it in the current working directory. Or, within Julia,

In [8]:
isfile("chr22.1kg.ref.phase1_release_v3.20101123.vcf.gz") || download("http://bochet.gcc.biostat.washington.edu/beagle/1000_Genomes_phase1_vcf/chr22.1kg.ref.phase1_release_v3.20101123.vcf.gz", 
    joinpath(pwd(), "chr22.1kg.ref.phase1_release_v3.20101123.vcf.gz"));
stat("chr22.1kg.ref.phase1_release_v3.20101123.vcf.gz")

StatStruct(mode=0o100644, size=51505515)

There are 294,622 markers and 1,092 individuals (or 2,184 haplotypes) in the reference panel.

In [9]:
using VCFTools

nrecords("chr22.1kg.ref.phase1_release_v3.20101123.vcf.gz"), nsamples("chr22.1kg.ref.phase1_release_v3.20101123.vcf.gz")

(294622, 1092)

First 10 liens of the reference panel VCF:

In [10]:
fh = openvcf("chr22.1kg.ref.phase1_release_v3.20101123.vcf.gz", "r")
for l in 1:10
    println(readline(fh))
end
close(fh)

##fileformat=VCFv4.1
##filedate=20130518
##source="simplfy-vcf 0.9"
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	HG00096	HG00097	HG00099	HG00100	HG00101	HG00102	HG00103	HG00104	HG00106	HG00108	HG00109	HG00110	HG00111	HG00112	HG00113	HG00114	HG00116	HG00117	HG00118	HG00119	HG00120	HG00121	HG00122	HG00123	HG00124	HG00125	HG00126	HG00127	HG00128	HG00129	HG00130	HG00131	HG00133	HG00134	HG00135	HG00136	HG00137	HG00138	HG00139	HG00140	HG00141	HG00142	HG00143	HG00146	HG00148	HG00149	HG00150	HG00151	HG00152	HG00154	HG00155	HG00156	HG00158	HG00159	HG00160	HG00171	HG00173	HG00174	HG00176	HG00177	HG00178	HG00179	HG00180	HG00182	HG00183	HG00185	HG00186	HG00187	HG00188	HG00189	HG00190	HG00231	HG00232	HG00233	HG00234	HG00235	HG00236	HG00237	HG00238	HG00239	HG00240	HG00242	HG00243	HG00244	HG00245	HG00246	HG00247	HG00249	HG00250	HG00251	HG00252	HG00253	HG00254	HG00255	HG00256	HG00257	HG00258	HG00259	HG00260	HG00261	HG00262	HG00263	HG00264	H

22	16050612	rs146752890	C	G	.	PASS	.	GT	0|1	0|1	0|1	0|0	0|1	0|0	0|0	0|1	0|0	0|0	0|1	0|0	0|0	0|1	0|1	0|0	0|0	0|0	0|0	0|1	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|1	0|1	0|0	0|0	0|1	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|1	0|0	0|0	0|1	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|0	0|0	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|1	0|0	0|1	0|1	0|0	0|1	0|0	0|1	0|0	0|1	0|1	0|0	0|1	0|1	0|1	0|0	0|0	0|1	0|0	0|1	0|0	0|1	0|1	0|1	0|0	0|1	0|1	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|1	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	

22	16051107	rs6518357	C	A	.	PASS	.	GT	0|0	0|1	0|1	0|0	0|1	0|1	0|0	0|1	0|0	0|0	0|1	0|1	0|0	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|1	0|1	0|0	0|0	0|1	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|1	0|0	0|0	0|1	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|1	0|1	0|1	0|0	0|1	0|1	0|0	0|1	0|1	0|1	0|0	0|1	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|1	0|1	0|1	0|1	0|1	0|0	0|0	0|1	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|0	0|0	0|1	0|1	0|1	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|

## Match GT data according to ID

`conformgt_by_id` function matches the markers (with GT field) in two VCF files according to marker ID.